In [1]:
from huggingface_hub import list_models

def list_most_popular(task: str):
    for rank, model in enumerate(
        list_models(filter=task, sort='downloads', direction=-1)):
        if rank == 5:
            break
        print(f"{model.id}, {model.downloads}\n")

list_most_popular('text-classification')

cardiffnlp/twitter-roberta-base-sentiment-latest, 88040464

mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis, 19454354

distilbert/distilbert-base-uncased-finetuned-sst-2-english, 8566164

smilegate-ai/kor_unsmile, 8449607

cardiffnlp/twitter-roberta-base-sentiment, 3061579



In [2]:
list_most_popular('summarization')

google-t5/t5-base, 4747701

google-t5/t5-small, 3790281

facebook/bart-large-cnn, 3447576

mrm8488/bert2bert_shared-spanish-finetuned-summarization, 834829

google-t5/t5-large, 552205



In [3]:
customer_email = """
I hope this email finds you amidst an aura of understanding, despite the tangled mess of emotions swirling within me as I write to you. I am writing to pour my heart out about the recent unfortunate experience I had with one of your coffee machines that arrived ominously broken, evoking a profound sense of disbelief and despair.

To set the scene, let me paint you a picture of the moment I anxiously unwrapped the box containing my highly anticipated coffee machine. The blatant excitement coursing through my veins could rival the vigorous flow of coffee through its finest espresso artistry. However, what I discovered within broke not only my spirit but also any semblance of confidence I had placed in your esteemed brand.

Imagine, if you can, the utter shock and disbelief that took hold of me as I laid eyes on a disheveled and mangled coffee machine. Its once elegant exterior was marred by the scars of travel, resembling a war-torn soldier who had fought valiantly on the fields of some espresso battlefield. This heartbreaking display of negligence shattered my dreams of indulging in daily coffee perfection, leaving me emotionally distraught and inconsolable
"""  # created by GPT-3.5

from transformers import pipeline

sentiment_model = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)

In [4]:
print(sentiment_model(customer_email))

[{'label': 'LABEL_0', 'score': 0.5822015404701233}]


In [11]:
from langchain import HuggingFaceHub

summarizer = HuggingFaceHub(
    repo_id='facebook/bart-large-cnn',
    model_kwargs={"temperature":0, "max_length": 180}
)
def summarize(llm, text) -> str:
    return llm(f"Summarize this: {text}!")

summarize(summarizer, customer_email)

/Users/taemin/anaconda3/envs/langchain_ai/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


'A customer\'s coffee machine arrived ominously broken, evoking a profound sense of disbelief and despair. "This heartbreaking display of negligence shattered my dreams of indulging in daily coffee perfection, leaving me emotionally distraught and inconsolable," the customer writes. "I hope this email finds you amidst an aura of understanding, despite the tangled mess of emotions swirling within me as I write to you," he adds.'

In [7]:
print(sentiment_model("I am so angry and sad, I want to kill myself!"))

[{'label': 'LABEL_0', 'score': 0.9788626432418823}]


In [8]:
print(sentiment_model("I am elated, I am so happy, this is the best thing that ever happened to me!"))

[{'label': 'LABEL_2', 'score': 0.9926880598068237}]


In [9]:
print(sentiment_model("I don't care. I guess it's ok, or not, I couldn't care one way or the other"))


[{'label': 'LABEL_1', 'score': 0.5958542823791504}]


In [10]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

template = """Given this text, decide what is the issue the customer is concerned about. Valid categories are these:
* product issues
* delivery problems
* missing or late orders
* wrong product
* cancellation request
* refund or exchange
* bad support experience
* no clear reason to be upset

Text: {email}
Category:
"""
prompt = PromptTemplate(template=template, input_variables=["email"])
llm = VertexAI()
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
print(llm_chain.run(customer_email))

GoogleAuthError: 
Unable to authenticate your request.
Depending on your runtime environment, you can complete authentication by:
- if in local JupyterLab instance: `!gcloud auth login` 
- if in Colab:
    -`from google.colab import auth`
    -`auth.authenticate_user()`
- if in service account or other: please follow guidance in https://cloud.google.com/docs/authentication